In [3]:
from match_tfidf_with_joblib import match_tfidf_with_joblib

match_tfidf_with_joblib(
    docs_joblib_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject/data/docs_tfidf.joblib",
    queries_joblib_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject/data/queries_tfidf.joblib",
    output_path=r"C:/Users/User/Desktop/PythonProject/results/tfidf_results_batch.json",
    top_k=100,
    batch_size_queries=100
)


In [5]:
# === مثال تنفيذ ===
if __name__ == "__main__":
    docs_joblib_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\TF-IDF\beir\quora\test\doc\tfidf_data.joblib"
    queries_joblib_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Query Processing\tfidf\queries_tfidf_reencoded.joblib"

    match_tfidf_with_joblib(
        docs_joblib_path=docs_joblib_path,
        queries_joblib_path=queries_joblib_path,
        top_k=100,
        batch_size_queries=100
    )


🔹 تحميل ملف الوثائق (tfidf_matrix + doc_ids)...
🔹 تحميل ملف الاستعلامات (query_tfidf_matrix + query_ids)...
📏 عدد الاستعلامات: 10000, عدد الوثائق: 522931


🔄 مطابقة دفعات الاستعلامات: 100%|██████████| 100/100 [08:40<00:00,  5.21s/it]


✅ تم حفظ نتائج المطابقة في tfidf_results_batch.json


In [26]:
import joblib
from scipy.sparse import csr_matrix

# تحميل النصوص الأصلية للاستعلامات
queries_data = joblib.load(r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Query Processing\tfidf\queries_tfidf.joblib")
query_ids = queries_data["query_ids"]
clean_texts = queries_data["clean_texts"]

# تحميل vectorizer الخاص بالوثائق
doc_vectorizer_data = joblib.load(r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\TF-IDF\beir\quora\test\doc\tfidf_data.joblib")
vectorizer = doc_vectorizer_data["vectorizer"]

# إعادة تحويل الاستعلامات لنفس فضاء الميزات
query_matrix = vectorizer.transform(clean_texts)

# حفظ التمثيل المعاد ترميزه
joblib.dump({
    "query_ids": query_ids,
    "clean_texts": clean_texts,
    "query_tfidf_matrix": query_matrix,
    "vectorizer": vectorizer  # اختياري للتوثيق
}, "queries_tfidf_reencoded.joblib")

print("✅ تم حفظ الاستعلامات بتمثيل متوافق مع الوثائق.")


✅ تم حفظ الاستعلامات بتمثيل متوافق مع الوثائق.


In [3]:
import joblib
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import json
from scipy.sparse import csr_matrix



def custom_tokenizer(text):
    return text.split()

def match_tfidf_with_joblib(
    docs_joblib_path: str,
    queries_joblib_path: str,
    top_k: int = 100,
    batch_size_queries: int = 100
):
    print("🔹 تحميل ملف الوثائق (tfidf_matrix + doc_ids)...")
    docs_data = joblib.load(docs_joblib_path)
    doc_ids = docs_data["doc_ids"]
    tfidf_docs: csr_matrix = docs_data["tfidf_matrix"]

    print("🔹 تحميل ملف الاستعلامات (query_tfidf_matrix + query_ids)...")
    queries_data = joblib.load(queries_joblib_path)

    # اختيار مصفوفة الـ tfidf الخاصة بالاستعلامات بطريقة سليمة
    if "query_tfidf_matrix" in queries_data:
        tfidf_queries = queries_data["query_tfidf_matrix"]
    elif "tfidf_matrix" in queries_data:
        tfidf_queries = queries_data["tfidf_matrix"]
    elif "queries_tfidf_matrix" in queries_data:
        tfidf_queries = queries_data["queries_tfidf_matrix"]
    else:
        raise ValueError("❌ لم يتم العثور على أي مصفوفة tfidf في ملف الاستعلامات.")

    # اختيار الـ query_ids بطريقة سليمة
    if "query_ids" in queries_data:
        query_ids = queries_data["query_ids"]
    elif "doc_ids" in queries_data:
        query_ids = queries_data["doc_ids"]
    else:
        raise ValueError("❌ لم يتم العثور على 'query_ids' أو 'doc_ids' في ملف الاستعلامات.")

    num_queries = tfidf_queries.shape[0]
    print(f"📏 عدد الاستعلامات: {num_queries}, عدد الوثائق: {tfidf_docs.shape[0]}")

    results = {}

    for start in tqdm(range(0, num_queries, batch_size_queries), desc="🔄 مطابقة دفعات الاستعلامات"):
        end = min(start + batch_size_queries, num_queries)
        batch_queries = tfidf_queries[start:end]

        # حساب مصفوفة التشابه (batch_size_queries x num_docs)
        sim_matrix = cosine_similarity(batch_queries, tfidf_docs)

        for i, query_idx in enumerate(range(start, end)):
            sims = sim_matrix[i]
            top_indices = np.argpartition(sims, -top_k)[-top_k:]
            top_scores = sims[top_indices]
            sorted_idx = top_indices[np.argsort(-top_scores)]

            results[query_ids[query_idx]] = [
                (doc_ids[idx], float(sims[idx])) for idx in sorted_idx
            ]

    # حفظ النتائج
    with open("tfidf_results_batch_antique.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)

    print("✅ تم حفظ نتائج المطابقة في tfidf_results_batch_antique.json")




In [6]:
# === مثال تنفيذ ===
if __name__ == "__main__":
    docs_joblib_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\TF-IDF\antique\train\doc\tfidf_data.joblib"
    queries_joblib_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Query Processing\tfidf\queries_tfidf_antique.joblib"

    match_tfidf_with_joblib(
        docs_joblib_path=docs_joblib_path,
        queries_joblib_path=queries_joblib_path,
        top_k=100,
        batch_size_queries=100
    )


🔹 تحميل ملف الوثائق (tfidf_matrix + doc_ids)...
🔹 تحميل ملف الاستعلامات (query_tfidf_matrix + query_ids)...
📏 عدد الاستعلامات: 176, عدد الوثائق: 403666


🔄 مطابقة دفعات الاستعلامات: 100%|██████████| 2/2 [00:04<00:00,  2.10s/it]

✅ تم حفظ نتائج المطابقة في tfidf_results_batch_antique.json
